# Introduction
* The notebook gives an introduction to NLP.
* The notebook also introduce us to some preprocessing techniques required for text data
* We will be working on the famous fake news dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

### Importing dataset

In [2]:
df = pd.read_csv('../input/fake-news/train.csv')
test = pd.read_csv('../input/fake-news/test.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Data preprocessing and cleaning

In [4]:
#filling NULL values with empty string
df=df.fillna('')
test=test.fillna('')

In [5]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']

In [6]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [7]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg = X.copy()
msg_test = test.copy()

In [8]:
#Downloading stopwords 
#Stopwords are the words in any language which does not add much meaning to a sentence.
#They can safely be ignored without sacrificing the meaning of the sentence.
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
#We will be using Stemming here
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [10]:
# Applying stemming and some preprocessing
for i in range(len(msg)):
    # Remove all non-alphabetic characters and replace them with spaces
    review = re.sub('[^a-zA-Z]', ' ', msg['total'][i])
    
    # Convert the text to lowercase
    review = review.lower()
    
    # Split the text into individual words
    review = review.split()
    
    # Apply stemming to each word and remove stopwords
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    
    # Join the processed words back into a single string
    review = ' '.join(review)
    
    # Append the processed review to the corpus
    corpus.append(review)


In [11]:
#Applying stemming and some preprocessing for test data
corpus_test = []
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [12]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [13]:
onehot_rep[:5]

[[1699, 214, 4089, 396, 2412, 3683, 307, 2686, 3996, 683, 258, 1691],
 [2593, 4257, 2665, 4887, 3875, 3505, 3837, 2264, 3600, 2593],
 [2487, 1565, 4288, 1159, 337, 3304],
 [680, 3153, 3464, 2914, 4729, 1658, 2220, 39],
 [4138, 3875, 2045, 15, 2483, 1948, 3875, 200, 2367, 4051, 2487, 2787]]

In [14]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [15]:
embedded_docs[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1699,  214, 4089,  396, 2412, 3683,  307, 2686, 3996,
         683,  258, 1691],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 2593, 4257, 2665, 4887, 3875, 3505, 3837,
        2264, 3600, 2593],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 2487, 1565, 4288,
        1159,  337, 3304],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  680, 3153, 3464, 2914, 4729,
        1658, 2220,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 4138, 3875, 2045,   15, 2483, 1948, 3875,  200, 2367,
        4051, 2487, 2787]], dtype=int32)

# Creating and training model

In [16]:
#We have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [17]:
#Converting into numpy array
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (5200, 25))

In [18]:
X_final[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1699,  214, 4089,  396, 2412, 3683,  307, 2686, 3996,
         683,  258, 1691],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 2593, 4257, 2665, 4887, 3875, 3505, 3837,
        2264, 3600, 2593],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 2487, 1565, 4288,
        1159,  337, 3304],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  680, 3153, 3464, 2914, 4729,
        1658, 2220,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 4138, 3875, 2045,   15, 2483, 1948, 3875,  200, 2367,
        4051, 2487, 2787]], dtype=int32)

In [19]:
#training model
model.fit(X_final,y_final,epochs=20,batch_size=64)

Epoch 1/20
325/325 [==============================] - 11s 28ms/step - loss: 0.3310 - accuracy: 0.8194
Epoch 2/20
325/325 [==============================] - 9s 27ms/step - loss: 0.0215 - accuracy: 0.9940
Epoch 3/20
325/325 [==============================] - 9s 27ms/step - loss: 0.0079 - accuracy: 0.9982
Epoch 4/20
325/325 [==============================] - 9s 27ms/step - loss: 0.0049 - accuracy: 0.9989
Epoch 5/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0036 - accuracy: 0.9988
Epoch 6/20
325/325 [==============================] - 9s 27ms/step - loss: 0.0022 - accuracy: 0.9993
Epoch 7/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0020 - accuracy: 0.9994
Epoch 8/20
325/325 [==============================] - 9s 29ms/step - loss: 0.0019 - accuracy: 0.9994
Epoch 9/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0013 - accuracy: 0.9996
Epoch 10/20
325/325 [==============================] - 9s 29ms/step - loss: 0.0021 - accur

## Creating Submission file 

In [20]:
y_pred = model.predict_classes(test_final)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [21]:
final_sub = pd.DataFrame()
final_sub['id']=test['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)

In [22]:
final_sub.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


# Conclusion
* LSTM with embedding layer works great
* The model gives more than 99% accuracy on test data.
* Furthermore we can also try vectoriztion or bi-directional LSTM.

![](https://st3.depositphotos.com/1998651/13850/v/600/depositphotos_138506364-stock-illustration-cup-of-coffee-with-have.jpg)